In [67]:
import pandas as pd
import numpy as np
import re
import string
import nltk
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\thesa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\thesa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [28]:
def remove_punctuation ( text ):
    return re.sub('[%s]' % re.escape(string.punctuation), ' ', text)

def tokenization(text):
    text = remove_punctuation(text)
    stemmer = SnowballStemmer('english')

    stopw = stopwords.words('english')
    stopw.remove('no')

    text_token = word_tokenize(text)
    for word in text_token:
        if word in stopw:
            text_token.remove(word)

    stemmed_text = ""
    for word in text_token:
        stemmed_text += " " + stemmer.stem(word)
    #stemmed_text = [stemmer.stem(word) for word in text_token]

    return stemmed_text

In [48]:
columns = ["labels", "reviews"]
data = []

def line_processing(line):
    line = line.replace('\n','')
    line = line.strip()
    parts = linea.split(':')

    if len(parts) != 2 or parts[1]=='\n':
        return None

    space = parts[0].find(' ')

    parts[1] = parts[0][space+1:]  + parts[1]
    parts[0] = parts[0].replace(parts[0][space:], '')
    parts[0] = parts[0].replace('__label__1', '0')
    parts[0] = parts[0].replace('__label__2', '1')
    parts[1] = tokenization(parts[1])
    return parts

pos_count = 0
neg_count = 0
ruta_txt = "train.ft.txt"
with open(ruta_txt, 'r', encoding='utf-8') as archivo:
    for linea in archivo:
        resultado = line_processing(linea)
        if resultado is not None:
            if resultado[0] == '0' and neg_count < 25000:
                neg_count += 1
                data.append(resultado)
            if resultado[0] == '1' and pos_count < 25000:
                pos_count += 1
                data.append(resultado)

            if pos_count >= 25000 and neg_count >= 25000:
                break


df = pd.DataFrame(data, columns=columns)
print(df)
print('Label 0:', (df['labels'] == '0').sum())
print('Label 1:', (df['labels'] == '1').sum())

      labels                                            reviews
0          1   stune even non gamer this sound track beauti ...
1          1   the best soundtrack ever anyth i read lot rev...
2          1   amaz this soundtrack favorit music time hand ...
3          1   absolut masterpiec i quit sure actual take ti...
4          0   buyer bewar this self publish book if want kn...
...      ...                                                ...
49995      0   not total happi one the cd arriv damag crack ...
49996      0   1 everi 4 defect i normal write review item i...
49997      0   poor pix qualiti well sell featur 10x optic z...
49998      0   onli 2 year last toy i bought camera panason ...
49999      0   take pictur indoor skip this camera i had cam...

[50000 rows x 2 columns]
Label 0: 25000
Label 1: 25000


In [55]:
def bag_of_words(textos):
    vectorizer = CountVectorizer()
    actual = []
    for i in range(len(textos)):
        actual.append(textos[i])

    X = vectorizer.fit_transform(actual)
    diccionario = vectorizer.get_feature_names_out()
    bow = X.toarray()
    return diccionario, bow

In [57]:
diccionario, X = bag_of_words(df['reviews'])

In [ ]:
logisticRegr = LogisticRegression()
logisticRegr.fit(X, df['labels'])

pred = logisticRegr.predict(X)
print("la precision de la regresion logistica es de: " + str(accuracy_score(pred, df['labels'])))
cm = confusion_matrix(df['labels'], pred, labels=logisticRegr.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=logisticRegr.classes_)
disp.plot()
plt.show()


KeyboardInterrupt

